In [ ]:
from google.colab import drive
from nltk.corpus import stopwords
import nltk
from collections import defaultdict
import re
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
import csv
import gc
import sys
csv.field_size_limit(sys.maxsize)

In [ ]:
drive.mount('/content/drive')
# going to our folder
% cd /content/drive/MyDrive/

In [ ]:
word_doc_index = {}
with open('./IROUTPUTS_v2/merged.txt') as f:
  next(f) # I have one empty line
  for line in f:
    word, *docs = line.strip().split(' ')
    word_doc_index[word] = tuple(docs) # tuple saves a little bit of memory since its read only

In [ ]:
del word_doc_index
gc.collect()

In [ ]:
doc_id_to_title = {}
shelf_to_title = defaultdict(list)
with open('./gutenberg_data.csv') as f:
  next(f)
  csv_file = csv.reader(f)
  for line in csv_file:
    title, author, link, id, bookshelf, text = line
    doc_id_to_title[id] = title

with open('./gutenberg_metadata.csv') as f:
  next(f)
  csv_file = csv.reader(f)
  for line in csv_file:
    title, author, link, bookshelf = line
    shelf_to_title[bookshelf].append(title)

In [ ]:
def remove_symbols(line):
    return re.sub('[^A-Za-z0-9\s]+', '', line).lower()
def preprocess_word(word):
  word = word.lower()
  return remove_symbols(porter.stem(word))
preprocess_word('happily')

In [ ]:
def query(comes_with=[], doesnt_come_with=[], shelf=None):
  all_docs = set()
  for word in comes_with:
    word = preprocess_word(word)
    docs = word_doc_index.get(word, [])
    # below line can check if it has atleast one word, then accept the doc
    # all_docs.update(set(docs)) # <= this line can work for unioning
    if all_docs:
      all_docs.intersection_update(set(docs)) # this operation will ensure that we want the doc to HAVE ALL THE WORDS
    else:
      all_docs.update(set(docs)) # in case we don't have anything in common
    if not all_docs:
      break
    # moreover we can even break if first for first N words there are no docs in common
  
  negation_docs = set()
  for word in doesnt_come_with:
    word = preprocess_word(word)
    docs = word_doc_index.get(word, [])
    negation_docs.update(set(docs))
  # removing the docs which have all terms which we dont want
  if negation_docs:
    all_docs.difference_update(set(docs)) # remove docs if they contain this word

  doc_titles = [doc_id_to_title[i] for i in all_docs]
  
  if shelf:
    shelf_docs = shelf_to_title.get(shelf, [])
    filtered_docs = [i for i in doc_titles if i in shelf_docs]
    return filtered_docs
  
  return doc_titles

# above step highly depends on what we are doing with word while storing
# moreover we can do mapping on these doc_ids too
query(comes_with=['fox', 'deer', 'human'], shelf='Animal')


matx = [title, result, bookshelf, author]
w = [wt, wr, wb, wa]
expected = [1 1 1 1 0]
expected =  w*matx
Ax = b
wt + wr + wb + wa = 1
text = 0.6
title, result, bookshelf, author = 0.1
w_text * text + w_title * title = result
if result > 0.5:
  result = 1
else:
  result = 0

text = 0.7, title = 0.3

A * [w_t w_ti w_a w_b] = y
x = A(-1) * y 